In [ ]:
# Decorrelation helper functions:
def cov(X, Y):
    return ((X - X.mean()) * (Y - Y.mean())).mean()

def cov_projection(X, Y, F=Cov):
    return X * (F(X, Y) / F(X, X))
 
def to_unit_variance(x, F=Cov):
    return x / np.sqrt(F(x, x))
 
def random_decorrelation(A, F=Cov):
    B = A.copy()
    n = A.shape[1]
    rng = default_rng()
    shuffled_indices = list(rng.permutation(n))
    for count, i_index in enumerate(shuffled_indices):
        for j in range(count):
            j_index = shuffled_indices[j]
            B[:, i_index] = B[:, i_index] - Proj(B[:, j_index], B[:, i_index], F=F)
        B[:, i_index] = to_unit_variance(B[:, i_index], F=F)
    return B

In [ ]:
corr_arr = []
for i in range(5):
    print(i)
    decorr_array = random_decorrelation(normed_array)
    corr_vector = []
    for col_idx in range(normed_array.shape[1]):
        rho, pval = stats.spearmanr(f91s_decision_values, decorr_array[:, col_idx])
        corr_vector.append(rho)
    corr_vector = np.stack(corr_vector)
    corr_arr.append(corr_vector)

In [ ]:
# Finding most discriminable primitive features:
# corr_arr = []
# for col_idx in range(normed_array.shape[1]):
#     rho, pval = stats.spearmanr(f91s_decision_values, normed_array[:, col_idx])
#     corr_arr.append(rho)
# corr_arr = np.stack(corr_arr)

# Retrieving most relevant features for LDA transform:
f91s_features = np.array(lda_feature_list)[std_mask][np.abs(summarised_corr_arr) > 0.05]
important_feature_corrs = summarised_corr_arr[np.abs(summarised_corr_arr) > 0.05]

# Plotting most relevant features:
fig, ax = plt.subplots(figsize=(5, 4), dpi=150)
feat_importances = pd.Series(important_feature_corrs, index=list(f91s_features))
feat_importances = feat_importances.sort_values()

feat_importances.plot(kind='barh', ax=ax)
ax.tick_params(axis='y', labelrotation = 15, labelsize=6)
plt.axvline(x=0, linewidth=1, color='k')

# ax.get_yticklabels()[-6].set_color("red")
# ax.get_yticklabels()[6].set_color("red")

# Titling figure:
plt.title(label="F91S Discriminant Axis - Highest Absolute Rank Correlations", fontsize=6);
plt.tight_layout()
plt.savefig(
    fname="M2 Mutant Analysis/Output Figures/F91S Discriminant Axis - Highest Absolute Rank Correlations.png", format="png"
)

In [ ]:
# Finding most discriminable primitive features:
corr_arr = []
for col_idx in range(normed_array.shape[1]):
    rho, pval = stats.spearmanr(r45h_decision_values, normed_array[:, col_idx])
    corr_arr.append(rho)
corr_arr = np.stack(corr_arr)

# Retrieving most relevant features for LDA transform:
r45h_features = np.array(lda_feature_list)[std_mask][np.abs(corr_arr) > 0.5]
important_feature_corrs = corr_arr[np.abs(corr_arr) > 0.5]

# Plotting most relevant features:
fig, ax = plt.subplots(figsize=(5, 4), dpi=150)
feat_importances = pd.Series(important_feature_corrs, index=list(r45h_features))
feat_importances = feat_importances.sort_values()

feat_importances.plot(kind='barh', ax=ax)
ax.tick_params(axis='y', labelrotation = 15, labelsize=6)
plt.axvline(x=0, linewidth=1, color='k')

# Highlighting Ch11 features:
ax.get_yticklabels()[3].set_color("red")
ax.get_yticklabels()[6].set_color("red")

# Titling figure:
plt.title(label="R45H Discriminant Axis - Highest Absolute Rank Correlations", fontsize=6);
plt.tight_layout()

plt.savefig(
    fname="M2 Mutant Analysis/Output Figures/R45H Discriminant Axis - Highest Absolute Rank Correlations.png", format="png"
)

In [ ]:
imp_feat = normed_array[:, np.abs(corr_arr) > 0.1]
corr_mat = np.corrcoef(imp_feat, rowvar=False)
sns.clustermap(np.abs(corr_mat))